# This is a new kind of Web Browser. 

## Give it a URL, and it will respond with a summary. 

# The Reader's Digest of the internet!!

In [2]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

## Connect to OpenAI API

In [3]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [4]:
openai = OpenAI()


In [5]:
# Check the API key by making a simple request

# To give you a preview -- calling OpenAI with these messages is this easy. Any problems, head over to the Troubleshooting notebook.

message = "Hello, GPT! This is my first ever message to you! Hi!"
response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

Hello! Welcome! I'm glad to hear from you. How can I assist you today?


## Project Code & Use of BeautifulSoup

In [6]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website: 

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)


In [7]:
# Let's try one out. Change the website and add print statements to follow along.

ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

Home - Edward Donner
Home
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connect
with me for

## Types of Prompts 

Models like GPT4o have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [8]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [9]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [10]:
print(user_prompt_for(ed))

You are looking at a website titled Home - Edward Donner
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Home
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.

## Messages & Sending Messages to OpenAI

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]
```

As a preview, the next 2 cells make a rather simple call - let's not stretch the mighty GPT (yet!)

In [11]:
messages = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

In [12]:
# To give you a preview -- calling OpenAI with system and user messages:

response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
print(response.choices[0].message.content)

Oh, just a mind-blowing 4. I know, right? Stop the presses!


## And now build useful messages for GPT-4o-mini, using a function

In [13]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [14]:
# Try this out, and then try for a few more websites

messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Home - Edward Donner\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nHome\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make a massive, posi

## Time to bring it together - And the API for OpenAI in action!

In [15]:
# And now: call the OpenAI API. 

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [16]:
summarize("https://edwarddonner.com")

'# Summary of Edward Donner\'s Website\n\nEdward Donner\'s website serves as a platform to share his interests and ventures, particularly in the field of AI and large language models (LLMs). Ed, who is the co-founder and CTO of Nebula.io, discusses his passion for coding, experimenting with LLMs, DJing, and electronic music production. \n\n## About Ed\n- Co-founder and CTO of Nebula.io, a company leveraging AI to enhance talent discovery and engagement.\n- Previously founded AI startup untapt, which was acquired in 2021.\n- His work involves proprietary LLMs tailored for talent management and he has a patented matching model.\n\n## Recent Announcements\n1. **January 23, 2025**: Resources for "LLM Workshop – Hands-on with Agents."\n2. **December 21, 2024**: A welcome message for "SuperDataScientists."\n3. **November 13, 2024**: Shared resources for "Mastering AI and LLM Engineering."\n4. **October 16, 2024**: Resources on "From Software Engineer to AI Data Scientist." \n\nThe website in

In [17]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [18]:
display_summary("https://edwarddonner.com")

# Summary of Edward Donner's Website

Edward Donner's website serves as an introduction to Ed, a technology enthusiast engaged in coding and experimenting with large language models (LLMs). He is the co-founder and CTO of Nebula.io, a company focused on leveraging AI to enhance talent discovery and management. His background includes founding the AI startup untapt, which was acquired in 2021. The site also features a section called "Outsmart," where LLMs engage in strategic competition.

## Recent Announcements
- **January 23, 2025**: Resources for a hands-on workshop on LLMs and agents.
- **December 21, 2024**: A welcome message for SuperDataScientists.
- **November 13, 2024**: Resources available for mastering AI and LLM engineering.
- **October 16, 2024**: Resources for transitioning from software engineering to AI data science.

# Let's try more websites

Note that this will only work on websites that can be scraped using this simplistic approach.

Websites that are rendered with Javascript, like React apps, won't show up. There are implementations in the community-contributions folder on github of ed donner for a Selenium implementation that gets around this. You'll need to read up on installing Selenium (ask ChatGPT!)

Also Websites protected with CloudFront (and similar) may give 403 errors.

But many websites will work just fine!

In [19]:
display_summary("https://cnn.com")

# Summary of CNN's Breaking News Website

The **CNN** website provides up-to-date news and videos covering a wide range of topics, including:

- **US and World News**: Covers significant local and international events.
- **Political Updates**: Includes analysis and live updates on political happenings, particularly surrounding prominent figures like Donald Trump and ongoing conflicts like the Ukraine-Russia war and the Israel-Hamas war.
- **Health and Lifestyle**: Discusses health topics, fitness, wellness, and more.
- **Entertainment and Sports**: Features stories about film, television, sports events, and notable personalities.

## Recent News Highlights
- Donald Trump asserts his mandate to eliminate wasteful spending and supports Elon Musk amid ongoing discussions regarding Musk's influence.
- A federal judge has blocked Musk's access to a critical Treasury payment system.
- Israeli forces have withdrawn from parts of Gaza as part of a ceasefire agreement, while concerns grow over the conditions of hostages released by Hamas.
- Various international stories include the tragic discovery of mass graves in Libya and President Modi's party regaining power in India's capital.

Overall, CNN's website serves as a comprehensive source for the latest breaking news across various sectors and offers live updates, in-depth analysis, and engaging video content.

In [20]:
display_summary("https://anthropic.com")

# Summary of Anthropic Website

**Overview:**
Anthropic is an AI safety and research company based in San Francisco, focused on developing AI systems with a strong emphasis on safety. The company has a multidisciplinary team with expertise in machine learning, physics, policy, and product development.

**Products:**
- **Claude:** The latest AI models include Claude 3.5 Sonnet and Claude 3.5 Haiku, designed for advanced interactions and applications.
- **API:** Offers developers the ability to create AI-powered applications using Claude.

**Recent Announcements:**
- On **October 22, 2024**, Anthropic introduced new capabilities, including computer use and two new models: Claude 3.5 Sonnet and Claude 3.5 Haiku.
- The company has also shared insights on "Constitutional AI" focusing on harmfulness reduction through AI feedback and their core views on AI safety.

**Research Focus:**
The company is committed to aligning AI technologies with safety and societal values, promoting research on AI beneficiality and safety measures.

**Careers:**
Anthropic offers various job opportunities for individuals interested in contributing to AI research and safety. 

For more details, visitors can explore various sections like the team, enterprise solutions, pricing, and additional research initiatives.